In [1]:
import sqlite3

In [10]:
import pandas as pd

In [29]:
import numpy as np

<br>
<br>

### Database

In [2]:
class SQLITE:

    def __init__(self):
        """
        
        """

    def connecting(self, databaseuri: str):
        """
        
        :param databaseuri: The URI of a '.db' database file
        """

        connection = None

        try:
            connection = sqlite3.connect(databaseuri)
        except ConnectionError as err:
            raise Exception(err.strerror) in err

        return connection

In [3]:
sqlite = SQLITE()

In [4]:
connection = sqlite.connecting(databaseuri='../data/gpinhours.sqlite')

In [5]:
c = connection.cursor()

In [6]:
inhours = c.execute('SELECT * FROM inhours').fetchall()

In [11]:
tables = c.execute("SELECT name FROM sqlite_schema WHERE type = 'table'").fetchall()

In [12]:
c.close()

In [13]:
connection.close()

<br>

Or

In [20]:
connection = sqlite.connecting(databaseuri='../data/gpinhours.sqlite')
query = 'SELECT * FROM inhours'
inhours = pd.read_sql_query(query, connection)
connection.close()

In [21]:
inhours.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47084 entries, 0 to 47083
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   CODE        47084 non-null  object 
 1   NAME        47084 non-null  object 
 2   TOTAL_POP   46862 non-null  object 
 3   Flu_OBS     46669 non-null  float64
 4   Vom_OBS     46672 non-null  float64
 5   Diarr_OBS   46672 non-null  float64
 6   Gastro_OBS  46757 non-null  float64
 7   lastdate    47084 non-null  float64
 8   week        47084 non-null  float64
dtypes: float64(6), object(3)
memory usage: 3.2+ MB


<br>
<br>

## Data Cleaning

In [22]:
inhours.head()

,CODE,NAME,TOTAL_POP,Flu_OBS,Vom_OBS,Diarr_OBS,Gastro_OBS,lastdate,week
0,E09000002,Barking and Dagenham,63009,1.0,13.0,9.0,33.0,17741.0,30.0
1,E09000003,Barnet,225813,1.0,40.0,101.0,148.0,17741.0,30.0
2,E09000004,Bexley,46339,1.0,11.0,12.0,24.0,17741.0,30.0
3,E09000005,Brent,178399,1.0,43.0,57.0,97.0,17741.0,30.0
4,E09000006,Bromley,168381,0.0,19.0,43.0,79.0,17741.0,30.0


<br>
<br>

### Fix the population count

<br>

**Create function**

In [24]:
def as_number_or_nan(x):
    return pd.to_numeric(x, errors='coerce')
    

In [27]:
inhours.loc[:, 'POP'] = as_number_or_nan(x = inhours.TOTAL_POP)

In [28]:
inhours.head()

,CODE,NAME,TOTAL_POP,Flu_OBS,Vom_OBS,Diarr_OBS,Gastro_OBS,lastdate,week,POP
0,E09000002,Barking and Dagenham,63009,1.0,13.0,9.0,33.0,17741.0,30.0,63009.0
1,E09000003,Barnet,225813,1.0,40.0,101.0,148.0,17741.0,30.0,225813.0
2,E09000004,Bexley,46339,1.0,11.0,12.0,24.0,17741.0,30.0,46339.0
3,E09000005,Brent,178399,1.0,43.0,57.0,97.0,17741.0,30.0,178399.0
4,E09000006,Bromley,168381,0.0,19.0,43.0,79.0,17741.0,30.0,168381.0


<br>

**Missing values**

In [34]:
inhours.loc[np.isnan(inhours.POP), 'NAME'].value_counts()

Bury                         115
Isle of Wight                 53
Wokingham                     50
Merton                         1
Greenwich                      1
Barking and Dagenham           1
Sutton                         1
Lambeth                        1
Cheshire West and Chester      1
Bexley                         1
Herefordshire, County of       1
Rochdale                       1
Knowsley                       1
Name: NAME, dtype: int64

<br>
<br>

### Fix the date

In [39]:
inhours.loc[:, 'date'] = pd.to_datetime(inhours.lastdate, unit='D', origin='unix')

In [40]:
inhours.date.describe(datetime_is_numeric=True)

count                            47084
mean     2017-01-17 20:39:29.620252928
min                2014-01-05 00:00:00
25%                2015-07-17 06:00:00
50%                2017-01-18 12:00:00
75%                2018-07-23 18:00:00
max                2020-01-26 00:00:00
Name: date, dtype: object

<br>
<br>

### Trim the years

In [42]:
inhours.loc[:, 'year'] = inhours.date.dt.year

In [43]:
inhours.year.value_counts()

2017    7897
2015    7748
2019    7748
2018    7748
2016    7748
2014    7599
2020     596
Name: year, dtype: int64

In [46]:
inhours.shape

(47084, 12)

<br>

Reduce

In [47]:
inhours = inhours.loc[inhours.year != 2020, :]

In [48]:
inhours.year.value_counts()

2017    7897
2015    7748
2019    7748
2018    7748
2016    7748
2014    7599
Name: year, dtype: int64

In [49]:
inhours.shape

(46488, 12)

<br>
<br>

### Drop zero populations

In [59]:
inhours = inhours.loc[inhours.POP != 0, :]

In [61]:
inhours = inhours.loc[~np.isnan(inhours.POP), :]

In [66]:
inhours

,CODE,NAME,TOTAL_POP,Flu_OBS,Vom_OBS,Diarr_OBS,Gastro_OBS,lastdate,week,POP,date,year
0,E09000002,Barking and Dagenham,63009,1.0,13.0,9.0,33.0,17741.0,30.0,63009.0,2018-07-29,2018
1,E09000003,Barnet,225813,1.0,40.0,101.0,148.0,17741.0,30.0,225813.0,2018-07-29,2018
2,E09000004,Bexley,46339,1.0,11.0,12.0,24.0,17741.0,30.0,46339.0,2018-07-29,2018
3,E09000005,Brent,178399,1.0,43.0,57.0,97.0,17741.0,30.0,178399.0,2018-07-29,2018
4,E09000006,Bromley,168381,0.0,19.0,43.0,79.0,17741.0,30.0,168381.0,2018-07-29,2018
...,...,...,...,...,...,...,...,...,...,...,...,...
46483,E06000041,Wokingham,150642,14.0,12.0,22.0,38.0,18258.0,52.0,150642.0,2019-12-28,2019
46484,E10000014,Hampshire,1284063,117.0,120.0,240.0,376.0,18258.0,52.0,1284063.0,2019-12-28,2019
46485,E06000046,Isle of Wight,67289,1.0,1.0,1.0,44.0,18258.0,52.0,67289.0,2019-12-28,2019
46486,E06000044,Portsmouth,183875,1.0,1.0,1.0,48.0,18258.0,52.0,183875.0,2019-12-28,2019


<br>
<br>

## North South Divide

<br>

### Read the table

In [63]:
connection = sqlite.connecting(databaseuri='../data/gpinhours.sqlite')
query = 'SELECT * FROM localauth'
localauth = pd.read_sql_query(query, connection)
connection.close()

In [64]:
localauth.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 149 entries, 0 to 148
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   NAME    149 non-null    object
 1   NS      149 non-null    object
dtypes: object(2)
memory usage: 2.5+ KB


In [65]:
localauth.head()

,NAME,NS
0,Barking and Dagenham,South
1,Barnet,South
2,Barnsley,North
3,Bath and North East Somerset,South
4,Bedford,South


<br>

### Merge with GP data

In [67]:
inhoursNS = inhours.merge(localauth, how='left', on='NAME')

In [68]:
inhoursNS

,CODE,NAME,TOTAL_POP,Flu_OBS,Vom_OBS,Diarr_OBS,Gastro_OBS,lastdate,week,POP,date,year,NS
0,E09000002,Barking and Dagenham,63009,1.0,13.0,9.0,33.0,17741.0,30.0,63009.0,2018-07-29,2018,South
1,E09000003,Barnet,225813,1.0,40.0,101.0,148.0,17741.0,30.0,225813.0,2018-07-29,2018,South
2,E09000004,Bexley,46339,1.0,11.0,12.0,24.0,17741.0,30.0,46339.0,2018-07-29,2018,South
3,E09000005,Brent,178399,1.0,43.0,57.0,97.0,17741.0,30.0,178399.0,2018-07-29,2018,South
4,E09000006,Bromley,168381,0.0,19.0,43.0,79.0,17741.0,30.0,168381.0,2018-07-29,2018,South
...,...,...,...,...,...,...,...,...,...,...,...,...,...
45919,E06000041,Wokingham,150642,14.0,12.0,22.0,38.0,18258.0,52.0,150642.0,2019-12-28,2019,South
45920,E10000014,Hampshire,1284063,117.0,120.0,240.0,376.0,18258.0,52.0,1284063.0,2019-12-28,2019,South
45921,E06000046,Isle of Wight,67289,1.0,1.0,1.0,44.0,18258.0,52.0,67289.0,2019-12-28,2019,South
45922,E06000044,Portsmouth,183875,1.0,1.0,1.0,48.0,18258.0,52.0,183875.0,2019-12-28,2019,South


<br>
<br>

### Group and aggregate

In [71]:
fields = ['NS', 'year', 'POP', 'Flu_OBS', 'Vom_OBS', 'Diarr_OBS', 'Gastro_OBS']

In [82]:
# Either
# inhoursNS[fields].groupby(by=['NS', 'year']).agg('sum')


# Or
aggNS = inhoursNS[fields].groupby(by=['NS', 'year']).agg(Total=('POP', sum), 
                                                         Flu_OBS=('Flu_OBS', sum), 
                                                         Vom_OBS=('Vom_OBS', sum), 
                                                         Diarr_OBS=('Diarr_OBS', sum), 
                                                         Gastro_OBS=('Gastro_OBS', sum))
aggNS

Total  Flu_OBS   Vom_OBS  Diarr_OBS  Gastro_OBS
NS    year                                                       
North 2014  719655399.0  31870.0  144469.0   279606.0    524322.0
      2015  812875983.0  44352.0  155056.0   302202.0    555413.0
      2016  852949550.0  48895.0  159742.0   307934.0    569276.0
      2017  737539944.0  30220.0  122787.0   248197.0    462994.0
      2018  607900399.0  46228.0   90951.0   194246.0    365863.0
      2019  614454668.0  31826.0   87114.0   186457.0    357594.0
South 2014  759999787.0  47093.0  141909.0   283547.0    507028.0
      2015  838254817.0  61180.0  147032.0   300360.0    537786.0
      2016  908035557.0  66822.0  156401.0   308993.0    558159.0
      2017  769928672.0  40572.0  117494.0   245726.0    442456.0
      2018  641245125.0  45791.0   84614.0   191328.0    343863.0
      2019  647648492.0  34353.0   79300.0   180489.0    324279.0

<br>
<br>

### Consultation rates